In [1]:
#import dependencies
import pandas as pd
import gmaps
import requests
#import the API key
from config import g_key


In [2]:
#store the csv you created in part one into a dataframe
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Tiksi,71.6872,128.8694,-1.03,93,100,14.79,RU,2021-02-02 21:57:41
1,1,Ushuaia,-54.8000,-68.3000,55.40,76,75,6.91,AR,2021-02-02 21:56:19
2,2,Caravelas,-17.7125,-39.2481,77.86,80,99,7.34,BR,2021-02-02 21:57:50
3,3,Kodiak,57.7900,-152.4072,30.20,93,90,15.95,US,2021-02-02 21:57:50
4,4,Port Elizabeth,-33.9180,25.5701,68.00,88,40,3.44,ZA,2021-02-02 21:57:50


In [3]:
#get the data types
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [4]:
#configure gmaps to use your google api key
gmaps.configure(api_key=g_key)

In [5]:
#heatmap of temperature
#get the latitude and longitude
locations = city_data_df[["Lat","Lng"]]
#get the max temperature
max_temp = city_data_df["Max Temp"]
#assign the figure variable
fig = gmaps.figure(center = (30.0,31.0), zoom_level=1.5)
#assign the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp], dissipating = False, max_intensity = 300, point_radius = 4)
#add the heatmap layer
fig.add_layer(heat_layer)
#call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
#heatmap of percent humidity
locations = city_data_df[["Lat","Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center = (30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity = 300, point_radius=4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"] 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"] 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [10]:
#filter the dataset to the cities that fit the criteria
perferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
perferred_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,Ushuaia,-54.8000,-68.3000,55.40,76,75,6.91,AR,2021-02-02 21:56:19
2,2,Caravelas,-17.7125,-39.2481,77.86,80,99,7.34,BR,2021-02-02 21:57:50
4,4,Port Elizabeth,-33.9180,25.5701,68.00,88,40,3.44,ZA,2021-02-02 21:57:50
5,5,Lompoc,34.6391,-120.4579,61.00,82,1,16.11,US,2021-02-02 21:57:50
6,6,Mataura,-46.1927,168.8643,64.17,71,99,3.87,NZ,2021-02-02 21:57:51


In [11]:
perferred_cities_df.count()

City_ID       350
City          350
Lat           350
Lng           350
Max Temp      350
Humidity      350
Cloudiness    350
Wind Speed    350
Country       347
Date          350
dtype: int64

In [26]:
perferred_cities_df.isnull().sum()
perferred_cities_df = perferred_cities_df.dropna()
perferred_cities_df.count()

City_ID       347
City          347
Lat           347
Lng           347
Max Temp      347
Humidity      347
Cloudiness    347
Wind Speed    347
Country       347
Date          347
dtype: int64

In [12]:
#create dataframe called hotel_df to store hotel names along with the city, country, max, temp, and coordinates
hotel_df = perferred_cities_df[["City","Country","Max Temp","Lat","Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Ushuaia,AR,55.40,-54.8000,-68.3000,
2,Caravelas,BR,77.86,-17.7125,-39.2481,
4,Port Elizabeth,ZA,68.00,-33.9180,25.5701,
5,Lompoc,US,61.00,34.6391,-120.4579,
6,Mataura,NZ,64.17,-46.1927,168.8643,


In [13]:
#set parameters to search for a hotel
params = {
    "radius" : 5000,
    "type" : "lodging",
    "key" : g_key
}

In [14]:
#iterate thru the dataframe
for index, row in hotel_df.iterrows():

    #get the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]

    #add the latitude and longitude to location key for the params dictionary. 
    params["location"] = f"{lat},{lng}"

    #use the search term: "lodging" and out latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    #make requests and get the json data from the search
    hotels = requests.get(base_url, params).json()
    
    #grab the first hotel from the results and stroe the name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
hotel_df.head(15)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Ushuaia,AR,55.40,-54.8000,-68.3000,Albatros Hotel
2,Caravelas,BR,77.86,-17.7125,-39.2481,Pousada dos Navegantes
4,Port Elizabeth,ZA,68.00,-33.9180,25.5701,39 On Nile Guest House
5,Lompoc,US,61.00,34.6391,-120.4579,Holiday Inn Express Lompoc
6,Mataura,NZ,64.17,-46.1927,168.8643,Ellie's Villa
8,Mahebourg,MU,80.60,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
9,Sai Buri,TH,74.66,6.7013,101.6168,ร้าน​ เวดดิ้ง
10,Monterey,US,55.99,36.2505,-121.2516,
11,Fare,PF,81.30,-16.7000,-151.0167,Maitai Lapita Village Huahine
12,Ghanzi,BW,69.89,-21.5667,21.7833,Ghanzi Farmhouse


In [27]:
#add a heatmp of temp for the vacation spots
locations = hotel_df[["Lat","Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0,31.0), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
#call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
# add a heatmp of temp for the vacation spots and marker for each city
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [30]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))